In [72]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.preprocessing.text import text_to_word_sequence

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt
# import matplotlib.pyplot

# NLTK
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
# from np import util # not working

# Word2Vec
# from gensim.test.utils import common_texts, get_tmpfile # not working
# from gensim.models import Word2Vec # not working

import re
from collections import Counter
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rogerleung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
# depends on which data set using
final_shuffled_csv = pd.read_csv("../final_downsampled.csv")

In [74]:
final_shuffled_csv.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,8K_Content,Adj Close_GSPC,Adj Close_stock,Close_GSPC,Close_stock,Date,...,Time,Time_of_day,Volume_GSPC,Volume_stock,Year,stock_overnight_movement_tag,8K_Content_cleaned,down,stay,up
0,2,71040,71317,112681,Check the appropriate box below ...,1372.18,9.10,1372.18,9.10,2012-02-28,...,2.012023e+13,out_of_market,3.579120e+09,4461800.0,2012.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
1,4,71914,72192,113869,Check the appropriate box below if the Form 8...,1460.91,24.93,1460.91,24.95,2012-10-17,...,2.012102e+13,out_of_market,3.655320e+09,10950000.0,2012.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
2,59,68493,68769,108833,Check the appropriate box below if the Form ...,1218.28,39.28,1218.28,40.04,2011-11-01,...,2.011110e+13,out_of_market,5.645540e+09,2762300.0,2011.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
3,75,63514,63779,100706,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,1472.29,20.56,1472.29,20.56,2007-09-05,...,2.007091e+13,out_of_market,2.991600e+09,2981100.0,2007.0,down,"['united', 'states', 'securities', 'and', 'exc...",1,0,0
4,86,7726,7806,12665,Table of Contents Check the appropriate ...,866.23,7.02,866.23,7.02,2009-04-24,...,2.009042e+13,out_of_market,7.114440e+09,445500.0,2009.0,down,"['table', 'of', 'contents', 'check', 'the', 'a...",1,0,0


In [75]:
import re
#cut to n words only

max_words = 1000

# from nltk.corpus import words
# word_list = words.words()



list_of_content = list(final_shuffled_csv["8K_Content"])
print(len(list_of_content))

# for i in range(len(list_of_content)):
#     content_split = list_of_content[i].split()
#     new_list = []
#     for j in range(len(content_split)):
#         print(j)
#         if content_split[j] in word_list:
#             new_list.append(content_split[j])
#     list_of_content[i] = ''.join(new_list)


for i in range(len(list_of_content)):
    if len(list_of_content[i].split()) >=max_words:
        list_of_content[i] = ' '.join(list_of_content[i].split()[:max_words])

# count_vectorization example
# from sklearn.feature_extraction.text import CountVectorizer

def better_preprocessor(s):
### STUDENT START ###    
    char_reg = re.compile('[`:\?!@_.0-9]') # remove numbers as well because in terms of topic 
                                         
    
    char_s = char_reg.sub(' ', s) #replace with space
    
    
    longword = re.compile(r'\W*\b\w{15,}\b') #remove long words
    char_s = longword.sub(' ', char_s)
    
    shortword = re.compile(r'\W*\b\w{1,3}\b') #remove 1 ~ 3 charcater words
    char_s = shortword.sub(' ', char_s)
            
    char_s = char_s.lower() #return lower case
            
    return  char_s #pass char_s to stemmer function

from sklearn.feature_extraction.text import CountVectorizer
corpus = list_of_content

vectorizer = CountVectorizer(preprocessor = better_preprocessor, max_features = 5000, stop_words = 'english')
X = vectorizer.fit_transform(corpus)

# create array
# arrays_of_words = X.toarray()
print(len(vectorizer.vocabulary_)) # Vocab Size


# create the dictionary 
vocab_dictionary = vectorizer.vocabulary_
vocab_size = len(vectorizer.vocabulary_)

# get the ids for each content

32250
5000


In [76]:
# testing most common words
features = vectorizer.get_feature_names()
print(features[:100])

['aaron', 'abaxis', 'abbott', 'abercrombie', 'ability', 'able', 'absence', 'absent', 'accelerate', 'accelerated', 'accelerating', 'acceleration', 'accenture', 'accept', 'acceptable', 'acceptance', 'accepted', 'access', 'accessed', 'accessible', 'accessing', 'accessories', 'accident', 'accompanied', 'accompanying', 'accomplished', 'accordance', 'according', 'accordingly', 'account', 'accountant', 'accountants', 'accounted', 'accounting', 'accounts', 'accretion', 'accretive', 'accrual', 'accruals', 'accrue', 'accrued', 'accumulated', 'accuracy', 'accurate', 'accurately', 'achieve', 'achieved', 'achievement', 'achievements', 'achieves', 'achieving', 'acknowledged', 'acquire', 'acquired', 'acquires', 'acquiring', 'acquisition', 'acquisitions', 'acreage', 'acres', 'acted', 'acthar', 'acting', 'action', 'actions', 'active', 'actively', 'activities', 'activity', 'acto', 'acts', 'actual', 'actually', 'actuant', 'acuity', 'acute', 'acxiom', 'adams', 'added', 'adding', 'addition', 'additional', 

In [77]:
# create the list of ids

content_8k_ids = []
counter = 0
for i in range(len(list_of_content)):
    vector = vectorizer.transform([list_of_content[i]])
    array = vector.toarray()
    ids = []
    count = 0
    for j in range(len(array[0])):
        if array[0][j] != 0:
            # number of times that word appears
            num_of_occurance = int(array[0][j])
            count += num_of_occurance 
            ids += [int(j+1)]*num_of_occurance # shift all the ids 1 because it's currently starting with 0
    if len(ids) >= max_words:
        ids = ids[:max_words]
    else:
        zeros_to_add = max_words - len(ids)
        ids += [0]*zeros_to_add
#     print(ids)
    content_8k_ids += [ids]
    counter += 1
    if counter == 500:
        print("500 records have been processed ", i)
        counter = 0

500 records have been processed  499
500 records have been processed  999
500 records have been processed  1499
500 records have been processed  1999
500 records have been processed  2499
500 records have been processed  2999
500 records have been processed  3499
500 records have been processed  3999
500 records have been processed  4499
500 records have been processed  4999
500 records have been processed  5499
500 records have been processed  5999
500 records have been processed  6499
500 records have been processed  6999
500 records have been processed  7499
500 records have been processed  7999
500 records have been processed  8499
500 records have been processed  8999
500 records have been processed  9499
500 records have been processed  9999
500 records have been processed  10499
500 records have been processed  10999
500 records have been processed  11499
500 records have been processed  11999
500 records have been processed  12499
500 records have been processed  12999
500 reco

In [78]:
df_for_model = pd.DataFrame(columns=['Year', '8K_ids', '8K_Content', 'down', 'stay', 'up', 'Ticker', 'Industry',"origainl_len","class"])

In [79]:
df_for_model['Year'] = final_shuffled_csv['Year'].apply(lambda x: int(x))
df_for_model['Ticker'] = final_shuffled_csv['Ticker']
df_for_model['Industry'] = final_shuffled_csv['Industry']
df_for_model['down'] = final_shuffled_csv['down']
df_for_model['stay'] = final_shuffled_csv['stay']
df_for_model['up'] = final_shuffled_csv['up']
df_for_model['8K_Content'] = final_shuffled_csv['8K_Content']
df_for_model['8K_ids'] = content_8k_ids
df_for_model['origainl_len'] = df_for_model['8K_ids'].apply(lambda x: len(x))
df_for_model['class'] = final_shuffled_csv['down']*0 + final_shuffled_csv['stay']*1 + final_shuffled_csv['up']*2

# def number_to_text(x):
#     if x == 0:
#         return "down"
#     elif x == 1:
#         return "stay"
#     else:
#         return "up"
    
# df_for_model['class'] = df_for_model['class'].apply(number_to_text)

In [80]:
df_for_model.head()

,Year,8K_ids,8K_Content,down,stay,up,Ticker,Industry,origainl_len,class
0,2012,"[85, 156, 207, 207, 297, 376, 406, 465, 465, 4...",Check the appropriate box below ...,1,0,0,CECO,Consumer Discretionary,500,0
1,2012,"[158, 250, 297, 357, 376, 376, 376, 406, 416, ...",Check the appropriate box below if the Form 8...,1,0,0,TXT,Industrials,500,0
2,2011,"[17, 27, 34, 57, 58, 156, 246, 246, 250, 250, ...",Check the appropriate box below if the Form ...,1,0,0,ABC,Health Care,500,0
3,2007,"[85, 166, 166, 207, 207, 207, 246, 250, 297, 3...",UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,1,0,0,OSIS,Information Technology,500,0
4,2009,"[17, 27, 27, 27, 34, 34, 73, 81, 83, 83, 199, ...",Table of Contents Check the appropriate ...,1,0,0,DGII,Information Technology,500,0


In [81]:
len(df_for_model["8K_ids"][0])

500

In [82]:
x_train = df_for_model['8K_ids'][df_for_model['Year'] <= 2008].values
x_dev = df_for_model['8K_ids'][(df_for_model['Year'] >= 2009) & 
                                       (df_for_model['Year'] <= 2010)].values
x_test = df_for_model['8K_ids'][df_for_model['Year'] >= 2011].values

y_train = df_for_model['class'][df_for_model['Year'] <= 2008].values
y_dev = df_for_model['class'][(df_for_model['Year'] >= 2009) & 
                                       (df_for_model['Year'] <= 2010)].values
y_test = df_for_model['class'][df_for_model['Year'] >= 2011]

ns_train = df_for_model["origainl_len"][df_for_model['Year'] <= 2008]
ns_dev = df_for_model["origainl_len"][(df_for_model['Year'] >= 2009) & 
                                       (df_for_model['Year'] <= 2010)]
ns_test = df_for_model["origainl_len"][df_for_model['Year'] >= 2011]

In [83]:
x_train = np.array(x_train.tolist())
x_dev = np.array(x_dev.tolist())
x_test = np.array(x_test.tolist())

# y_train = np.array(y_train.tolist())
# y_dev = np.array(y_dev.tolist())
# y_test = np.array(y_test.tolist())

# ns_train = np.array(ns_train.tolist())
# ns_dev = np.array(ns_dev.tolist())
# ns_test = np.array(ns_test.tolist())

In [84]:
len(x_train), len(x_dev), len(x_test)

(16835, 7577, 7838)

In [85]:
x_train.shape

(16835, 500)

In [86]:
ns_train_non_zero_len = []
for i in range(len(x_train)):
    number_of_non_zero = 0
    for j in range(len(x_train[i])):
        if int(x_train[i][j]) != 0:
            number_of_non_zero += 1
    ns_train_non_zero_len = ns_train_non_zero_len + [number_of_non_zero]
    
print(len(ns_train_non_zero_len))
            
ns_dev_non_zero_len = []
for i in range(len(x_dev)):
    number_of_non_zero = 0
    for j in range(len(x_dev[i])):
        if int(x_dev[i][j]) != 0:
            number_of_non_zero += 1
    ns_dev_non_zero_len = ns_dev_non_zero_len + [number_of_non_zero]
    
print(len(ns_dev_non_zero_len))

ns_test_non_zero_len = []
for i in range(len(x_test)):
    number_of_non_zero = 0
    for j in range(len(x_test[i])):
        if int(x_test[i][j]) != 0:
            number_of_non_zero += 1
    ns_test_non_zero_len = ns_test_non_zero_len + [number_of_non_zero]
    
print(len(ns_test_non_zero_len))

16835
7577
7838


In [87]:
ns_train = np.array(ns_train_non_zero_len)
ns_dev = np.array(ns_dev_non_zero_len)
ns_test = np.array(ns_test_non_zero_len)

In [88]:
# run neural BoW

from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML


# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
# assert(tf.__version__.startswith("1.8"))

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst, models, models_test

# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.


In [89]:
# import models
reload(models)
utils.run_tests(models_test, ["TestLayerBuilders", "TestNeuralBOW"])

test_embedding_layer (models_test.TestLayerBuilders) ... ok
test_softmax_output_layer (models_test.TestLayerBuilders) ... ok
test_BOW_encoder (models_test.TestNeuralBOW) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.086s

OK


In [90]:
import models; reload(models)

x, ns, y = x_train, ns_train, y_train
batch_size = 32

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab_size+1, embed_dim=300, hidden_dims=[10], num_classes=len([0,1,2]),
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)
model_fn = models.classifier_model_fn

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
with tf.Graph().as_default(), tf.Session() as sess:
    ##
    # Construct the graph here. No session.run calls - just wiring up Tensors.
    ##
    # Add placeholders so we can feed in data.
    x_ph_  = tf.placeholder(tf.int32, shape=[None, x.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    ##
    # Done constructing the graph, now we can make session.run calls.
    ##
    sess.run(tf.global_variables_initializer())
    
    # Run a single epoch
    t0 = time.time()
    for (bx, bns, by) in utils.multi_batch_generator(batch_size, x, ns, y):
        # feed NumPy arrays into the placeholder Tensors
        feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
        batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 25 == 0):
            print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
    print("Completed one epoch in {:s}".format(utils.pretty_timedelta(since=t0)))

  800 examples, moving-average loss 1.03
1,600 examples, moving-average loss 0.27
2,400 examples, moving-average loss 0.17
3,200 examples, moving-average loss 0.14
4,000 examples, moving-average loss 0.12
4,800 examples, moving-average loss 0.11
5,600 examples, moving-average loss 0.69
6,400 examples, moving-average loss 0.20
7,200 examples, moving-average loss 0.11
8,000 examples, moving-average loss 0.10
8,800 examples, moving-average loss 0.09
9,600 examples, moving-average loss 0.08
10,400 examples, moving-average loss 0.08
11,200 examples, moving-average loss 0.07
12,000 examples, moving-average loss 0.14
12,800 examples, moving-average loss 0.10
13,600 examples, moving-average loss 0.08
14,400 examples, moving-average loss 0.08
15,200 examples, moving-average loss 0.08
16,000 examples, moving-average loss 0.07
16,800 examples, moving-average loss 0.07
Completed one epoch in 0:00:14


In [91]:
import models; reload(models)

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab_size+1, embed_dim=300, hidden_dims=[10], num_classes=len([0,1,2]),
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
# Write vocabulary to file, so TensorBoard can label embeddings.
# creates checkpoint_dir/projector_config.pbtxt and checkpoint_dir/metadata.tsv
# ds.vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed") # comment out this one 

model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20181209-0653', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f366853ab00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20181209-0653' --port 6006

Then in your browser, open: http://localhost:6006


In [ ]:
# Training params, just used in this cell for the input_fn-s
sess = tf.Session()
train_params = dict(batch_size=32, total_epochs=5000, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.
# Input function for training set batches
# Do 'eval_every' epochs at once, followed by evaluating on the dev set.
# NOTE: use patch_numpy_io.numpy_input_fn instead of tf.estimator.inputs.numpy_input_fn
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": x_train, "ns": ns_train}, y=y_train,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

# Input function for dev set batches. As above, but:
# - Don't randomize order
# - Iterate exactly once (one epoch)
dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": x_dev, "ns": ns_dev}, y=y_dev,
                    batch_size=128, num_epochs=5, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on dev
    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181209-0653/model.ckpt-32643
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 32643 into /tmp/tf_bow_sst_20181209-0653/model.ckpt.
INFO:tensorflow:loss = 0.13467713, step = 32644
INFO:tensorflow:global_step/sec: 36.258
INFO:tensorflow:loss = 0.3299526, step = 32744 (2.759 sec)
INFO:tensorflow:global_step/sec: 35.8876
INFO:tensorflow:loss = 0.117044434, step = 32844 (2.786 sec)
INFO:tensorflow:global_step/sec: 35.8956
INFO:tensorflow:loss = 0.18914545, step = 32944 (2.786 sec)
INFO:tensorflow:global_step/sec: 36.1321
INFO:tensorflow:loss = 0.61677545, step = 33044 (2.768 sec)
INFO:tensorflow:global_step/sec: 35.8889
INFO:tensorflow:loss = 0.19020617, step = 33144 (2.786 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:global_step/sec: 36.4065
INFO:tensorflow:loss = 0.29094195, step = 36803 (2.747 sec)
INFO:tensorflow:Saving checkpoints for 36855 into /tmp/tf_bow_sst_20181209-0653/model.ckpt.
INFO:tensorflow:Loss for final step: 0.290832.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-09-07:29:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181209-0653/model.ckpt-36855
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-09-07:30:06
INFO:tensorflow:Saving dict for global step 36855: accuracy = 0.35343805, cross_entropy_loss = 2.6728628, global_step = 36855, loss = 2.924643
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 36855: /tmp/tf_bow_sst_20181209-0653/model.ckpt-36855
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Ch

INFO:tensorflow:global_step/sec: 35.4579
INFO:tensorflow:loss = 0.32946494, step = 40115 (2.822 sec)
INFO:tensorflow:global_step/sec: 35.9462
INFO:tensorflow:loss = 0.1193428, step = 40215 (2.782 sec)
INFO:tensorflow:global_step/sec: 36.2854
INFO:tensorflow:loss = 0.18790515, step = 40315 (2.756 sec)
INFO:tensorflow:global_step/sec: 36.6362
INFO:tensorflow:loss = 0.6692353, step = 40415 (2.730 sec)
INFO:tensorflow:global_step/sec: 36.4633
INFO:tensorflow:loss = 0.19119549, step = 40515 (2.743 sec)
INFO:tensorflow:global_step/sec: 36.4855
INFO:tensorflow:loss = 0.36443478, step = 40615 (2.740 sec)
INFO:tensorflow:global_step/sec: 36.8267
INFO:tensorflow:loss = 0.12513328, step = 40715 (2.715 sec)
INFO:tensorflow:global_step/sec: 36.5588
INFO:tensorflow:loss = 0.22164848, step = 40815 (2.735 sec)
INFO:tensorflow:global_step/sec: 36.4282
INFO:tensorflow:loss = 0.11154602, step = 40915 (2.745 sec)
INFO:tensorflow:global_step/sec: 36.1593
INFO:tensorflow:loss = 0.30300283, step = 41015 (2.7

In [110]:
# Evaluation

test_input_fn = tf.estimator.inputs.numpy_input_fn(  # replace with an input_fn, similar to dev_input_fn
                    x={"ids": x_test, "ns": ns_test}, y=y_test,
                    batch_size=128, num_epochs=1, shuffle=False
                )

from sklearn.metrics import accuracy_score
predictions = list(model.predict(test_input_fn))  # list of dicts
y_pred = [p['max'] for p in predictions]
y_pred_original = [p for p in predictions]
acc = accuracy_score(y_pred, y_test)
print("Accuracy on test set: {:.02%}".format(acc))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181209-0653/model.ckpt-32643
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


/home/rogerleung/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:841: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



Accuracy on test set: 32.76%


In [111]:
y_pred_original

[{'proba': array([0.93702704, 0.00369807, 0.05927492], dtype=float32),
  'max': 0},
 {'proba': array([0.98584855, 0.00393629, 0.0102151 ], dtype=float32),
  'max': 0},
 {'proba': array([0.8897012 , 0.00276044, 0.10753841], dtype=float32),
  'max': 0},
 {'proba': array([0.9653366 , 0.00186245, 0.03280105], dtype=float32),
  'max': 0},
 {'proba': array([0.95367104, 0.00582105, 0.04050784], dtype=float32),
  'max': 0},
 {'proba': array([0.98912746, 0.00255894, 0.00831358], dtype=float32),
  'max': 0},
 {'proba': array([0.97962934, 0.003228  , 0.01714267], dtype=float32),
  'max': 0},
 {'proba': array([0.57603264, 0.01298172, 0.41098565], dtype=float32),
  'max': 0},
 {'proba': array([0.9039388 , 0.00415228, 0.09190903], dtype=float32),
  'max': 0},
 {'proba': array([0.95418364, 0.00233069, 0.04348566], dtype=float32),
  'max': 0},
 {'proba': array([0.974589  , 0.00216673, 0.02324432], dtype=float32),
  'max': 0},
 {'proba': array([0.8724991 , 0.02719164, 0.10030922], dtype=float32),
  'ma

In [112]:
from collections import Counter

print(Counter(y_test))
print(Counter(y_pred))

df_for_evaluation = pd.DataFrame(columns=['Predicted', 'Actual'])

df_for_evaluation['Predicted'] = list(y_pred)
df_for_evaluation['Actual'] = list(y_test)

Counter({1: 2745, 0: 2572, 2: 2521})
Counter({0: 7749, 2: 74, 1: 15})


In [113]:
df_for_evaluation[df_for_evaluation["Predicted"] == 1 ]

,Predicted,Actual
893,1,0
1178,1,0
1187,1,0
1876,1,0
1897,1,0
2041,1,0
2181,1,0
2991,1,1
3940,1,1
4004,1,1


In [114]:
df_for_evaluation["results"] = df_for_evaluation["Predicted"] == df_for_evaluation["Actual"]

In [115]:
len(df_for_evaluation[(df_for_evaluation["Actual"] == 0)])

2572

In [99]:
len(df_for_evaluation)

7838